In [20]:
from typing import Callable
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier


def prior(df, household_idx=1):
    if household_idx in [1, 3]:
        df["preds_time"] = df.apply(lambda row: 0 if (row.Time >= 18) & (row.Time <= 40)  else 1)
    elif household_idx in [2]:
        df["preds_time"] = df.apply(lambda row: 0 if (row.Time >= 24) & (row.Time <= 28) else 1)

def multiview_iteration(df:pd.DataFrame, prior: Callable, max_iter:int=100):
    # line1
    df = prior(df)
    df["preds_t"] = df["preds_time"]
    
    stop_condition = False
    t = 1
    eta_1 = 0.5
    
    while (not stop_condition)&(t < max_iter):
        df_new = pd.DataFrame()
        
        # line2-5
        model_energy = DecisionTreeClassifier()
        model_energy.fit(df.Energy, df.preds)
        df["preds_energy"] = model_energy(df.Energy)
        
        # line6
        df["preds_majority_vote"] = df.preds_energy & df.preds_time
        
        # line 7
        df_new["preds_new"] = df[df.preds_t != df.preds_majority_vote]